In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
import torch

import os 
import dotenv
dotenv.load_dotenv() 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# embedding_model = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'), model="text-embedding-3-small")
embedding_model = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5', model_kwargs = {'device': device})

c:\Users\Admin\go_pro\finance_chatbot\group_2_chatbot_financial_statement\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_chroma import Chroma

## Requirement
- Database 
- Vector DB for your own db

In [7]:
from chromadb import Client, PersistentClient
from chromadb.config import Settings

client = PersistentClient(path = 'data/vector_db_vertical_local', settings = Settings())
company_name_chroma = Chroma(client = client, embedding_function=embedding_model, collection_name='company_name_chroma')

2024-12-13 22:21:35,837 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


`company_name_chroma`: ChromaDB Client on langchain

In [8]:
from ETL.dbmanager import BaseDBHUB

class NEWDBHUB(BaseDBHUB):
    def __init__(self, db_config, chroma):
        super().__init__(db_config)
        self.chroma = chroma

    def get_company_name_chroma(self, company_name):
        return self.chroma.get_vector(company_name)

In [1]:
text = """
-- ### financial_ratio

SELECT 
    fr.stock_code,
    fr.ratio_code,
    ROUND(fr.data::numeric * 100, 2) AS ratio_percentage
FROM 
    financial_ratio fr
WHERE 
    fr.stock_code = 'BID'
    AND fr.year = 2020
    AND fr.quarter = 0
    AND fr.ratio_code = 'ROA'
LIMIT 1

"""

In [8]:
import re
def get_table_name_from_sql(sql_text):
    pattern = r"-- ###\s*(.+)"
    matches = re.findall(pattern, sql_text)
    if len(matches) > 0:
        return matches[0]
    return ""

In [9]:
get_table_name_from_sql(text)

'financial_ratio'